In [1]:
import urllib.request, json 
from pandas.io.json import json_normalize
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import re
client_id = '614_33285ony1la8000k80wko00c8kw0c4c4sckoc0o4ck8w8owcow'
client_secret = '2vf9dmzc4x4wokcw0wssgs0ccsws8w0w4wg8swcc008co8o0k0'

In [2]:
# 1---------- generate access token 
url_accesstoken = 'https://api.coinmarketcal.com/oauth/v2/token?grant_type=client_credentials&client_id='+client_id+'&client_secret='+client_secret
with urllib.request.urlopen(url_accesstoken) as url:
    access_token = json.loads(url.read().decode())['access_token']

In [27]:
categories_url = 'https://api.coinmarketcal.com/v1/categories?access_token='+access_token
with urllib.request.urlopen(categories_url) as url:
    categories = json.loads(url.read().decode())

In [45]:
categories_df = pd.DataFrame()
for i in categories:
    categories_df = categories_df.append(i, ignore_index=True)
categories_df.columns = ['category_id','category_name']
#--------- write categories_df to db

In [35]:
categories_id = [i['id'] for i in categories]
categories_id_list = ''
for i in categories_id:
    categories_id_list = categories_id_list + '%2C'+str(i)
categories_id_list = categories_id_list[3:]

In [4]:
# 2. -------- get a list of all coins
url_coins = 'https://api.coinmarketcal.com/v1/coins?access_token='+access_token
with urllib.request.urlopen(url_coins) as url:
    coins = json.loads(url.read().decode())
coins_df = pd.DataFrame.from_dict(coins)
# - write coin_df to db

In [145]:
#--------------- get a list of 5 coins patch
coin_list = []
for k in range(5,coin_len,5):
    tmp = ''
    for l in range(k-5,k):
        tmp = tmp+'%2C'+coins_df.id.values[l]
    coin_list.append(tmp[3:])
    
#--- loop through each 5 coins and get all events
events_df = pd.DataFrame()
start_date ='11%2F01%2F2017'
end_date = '10%2F10%2F2020'
for c in coin_list:
    url_events = 'https://api.coinmarketcal.com/v1/events?access_token='+access_token+'&page=1&max=150&dateRangeStart='+start_date+'&dateRangeEnd='+end_date+'&coins='+c+'&categories='+categories_id_list+'&showOnly=hot_events'
    with urllib.request.urlopen(url_events) as url:
        events = json.loads(url.read().decode())
    if(len(events)>0):
        events_df_inner = pd.DataFrame(columns=[e for e in events[0].keys()])
        for j in events:
            j['coins']=j['coins'][0]['id']
            j['categories'] = j['categories'][0]['name']
            events_df_inner = events_df_inner.append(j,ignore_index=True)
        events_df = events_df.append(events_df_inner)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [148]:
#events_df_bk = events_df

In [222]:
events_df = events_df_bk.copy()

In [244]:
events_df.date_event = events_df['date_event'].apply(lambda x: datetime.date(x))
events_df.created_date = events_df['created_date'].apply(lambda x: datetime.date(x))

#------------- GOOD TO WRITE TO BD as EVENT table

In [245]:
events_df_short = events_df[['id','coins','title','date_event','created_date',
                            'percentage','can_occur_before','description']]

In [252]:
today = datetime.now().date()
events_df_short = events_df_short.query('percentage > 70 & date_event > @today')


In [255]:
events_df_short['events'] = events_df_short.title + " | " + events_df_short.description

In [259]:
events_df_short[['date_event','id','events']].pivot(index='id', columns='date_event', values='events')

date_event,2018-05-12,2018-05-13,2018-05-14,2018-05-15,2018-05-17,2018-05-20,2018-05-22,2018-05-24,2018-05-27,2018-05-28,...,2018-07-08,2018-07-29,2018-07-30,2018-08-12,2018-08-29,2018-08-30,2018-09-01,2018-09-22,2018-09-29,2018-09-30
id,,,,,,,,,,,,,,,,,,,,,
914,None,None,None,None,NaN,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1103,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,ClearPoll Official Launch | Full public launch...,None,None,None,None
2428,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,"Network Upgrade 1 | ""Sapling will activate the...",None
2429,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2800,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2857,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3211,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,NaN,None,None,None,None,None
3212,None,None,None,None,None,None,None,None,None,None,...,None,None,None,NaN,None,None,None,None,None,None
3214,None,None,None,None,None,None,None,None,None,None,...,None,NaN,None,None,None,None,None,None,None,None


In [262]:
events_df_short.to_csv('events_df_short.csv')